## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [1]:
#Importing helpful Libraries
import imageio
import os
import pandas as pd
import numpy as np
from skimage.transform import resize
import tensorflow as tf
from sklearn.model_selection import train_test_split
import random
from scipy import ndarray
import skimage as sk
from skimage import transform
from skimage import util

ImportError: cannot import name '_validate_lengths'

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [ ]:
from google.colab import drive
drive.mount('../content/gdrive')

In [ ]:
project_path = "/content/gdrive/My Drive/AIML-GL/Project/CNN/Project2/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [ ]:
from zipfile import ZipFile
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [ ]:
labels_df = pd.read_csv('labels.csv')

In [ ]:
labels_df.head(4)

In [ ]:
labels_df.shape

### Print the count of each category of Dogs given in the dataset



In [ ]:
labels_df.groupby(by='breed').agg({'id':'count'})

### Get one-hot encodings of labels

In [ ]:
dogbreeds = labels_df.breed.unique()

In [ ]:
# Label encoding
class_label = {}
index = 0
for breed in dogbreeds:
    class_label[breed] = index
    index += 1
class_label

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [ ]:
img_df = {'image': [],'label': []}

for data in labels_df.values:
    filename = os.path.join('train', data[0]+'.jpg')
    breedname = data[1]
    im = imageio.imread(filename)
    img = resize(im, (64, 64, 3))
    img_df['image'].append(img)
    img_df['label'].append(class_label[breedname])

In [ ]:
x_train = np.array(img_df['image'])
y_train = img_df['label']

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [ ]:
x_train = x_train.astype('float32')
#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [ ]:
x_train_data, x_test_data, y_train_data, y_test_data = train_test_split(x_train, y_train, test_size=0.20, random_state=1)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [ ]:
test_img = pd.read_csv('sample_submission.csv')

In [ ]:
test_img.head()

Run the below code to load the test image files in x_test_feature

In [ ]:
x_test_feature = []

for data in test_img.id:
    filename = os.path.join('test', str(data)+'.jpg')
    im = imageio.imread(filename)
    img = resize(im, (64, 64, 3))
    x_test_feature.append(img)

Normalize the test data and convert it into 4 dimensions

In [ ]:
x_test_feature = (x_test_feature-mean)/(std+1e-7)

In [ ]:
x_test_feature.shape

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [ ]:
import keras
from keras.models import Sequential
from keras import regularizers
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Dense, Activation, Dropout, Flatten, Reshape, BatchNormalization, Input
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
x_train_data.shape[1:]

In [ ]:
weight_decay = 1e-4

conv_layers = [
    Conv2D(128, (5,5), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=(64, 64, 3)),
    Activation('relu'),
    BatchNormalization(),
    Conv2D(128, (3, 3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)),
    Activation('relu'),
    BatchNormalization(),
    MaxPooling2D(pool_size=(2,2))
]
dense_layers = [
    Flatten(),
    Dense(256),
    Activation('relu'),
    BatchNormalization(),
    Dense(120),
    Activation('softmax')
]

model = Sequential(conv_layers + dense_layers)
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

### Use batch_size = 128 and epochs = 10 and execute the model

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10

model.fit(x_train_data,
          y_train_data, 
          epochs=EPOCHS,
          batch_size=BATCH_SIZE,
          verbose=True,
          validation_data=(x_test_data, y_test_data),
          callbacks=callback_list
)

In [ ]:
model.evaluate(x_test_data, y_test_data)

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [ ]:
breed_df = labels_df.groupby(by='breed').agg({'id':'count'})
max(breed_df.id)

In [ ]:
MAX_DATA = 125

img_df = {'image': [],'label': []}

for index, breed in breed_df.itertuples():
    nooffiles = breed
    breedname = index
    nooffiles_togen = MAX_DATA - nooffiles
    breed_data = labels_df[labels_df.breed == breedname]
    gen_files = 0
    for data in breed_data.values:
        filename = os.path.join('train', data[0]+'.jpg')
        im = imageio.imread(filename)
        img = resize(im, (128, 128, 3))
        img_df['image'].append(img)
        img_df['label'].append(class_label[breedname])
        if gen_files < nooffiles_togen:
            #Randon Rotation
            random_degree = random.uniform(-25, 25)
            transformed_image = sk.transform.rotate(img, random_degree)
            img_df['image'].append(transformed_image)
            img_df['label'].append(class_label[breedname])
            gen_files += 1
            #Horizontal Flip
            transformed_image = img[:, ::-1]
            img_df['image'].append(transformed_image)
            img_df['label'].append(class_label[breedname])
            gen_files += 1

In [ ]:
len(img_df['label'])

In [ ]:
pd.DataFrame(img_df['label']).groupby(by=0).agg({0:'count'})

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [ ]:
X = np.array(img_df['image'])
Y = img_df['label']

X = X.astype('float32')
mean = np.mean(X,axis=(0,1,2,3))
std = np.std(X,axis=(0,1,2,3))
X = (X-mean)/(std+1e-7)

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=1)

In [ ]:
train_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

train_generator.fit(x_train)

In [ ]:
value_generator = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=True
)

value_generator.fit(x_test)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [ ]:
BATCH_SIZE = 500
EPOCHS = 10

model.fit_generator(
    train_generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=EPOCHS*len(x_train)/BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    shuffle=True,
    validation_data=value_generator.flow(x_test, y_test),
    validation_steps=EPOCHS*len(x_test)/BATCH_SIZE,
    callbacks=callback_list
)

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [ ]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [ ]:
base_model.summary()

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [ ]:
base_model.output_shape[1:]

In [ ]:
dense_layers = [
#    Flatten(input_shape=(base_model.output_shape[1:])),
    Dense(1024),
    Activation('relu'),
    Dense(256),
    Activation('relu'),
    Dense(120),
    Activation('softmax')
]

new_model = Sequential()
for layer in base_model.layers:
    new_model.add(layer)
for layer in dense_layers:
    new_model.add(layer)

In [ ]:
#Flatten LAyer
base_model.add_update(Flatten())
# Fully Connected Layer 1
base_model.add_update(Dense(1024))
base_model.add_update(Activation('relu'))
# Fully Connected Layer 2
base_model.add_update(Dense(256))
base_model.add_update(Activation('relu'))

# Prediction Layer
base_model.add_update(Dense(output_dim=120, init='he_normal', bias=True))
base_model.add_update(Activation('softmax'))

In [ ]:
new_model.summary()

### Make all the layers in the base_model (VGG16) to be non-trainable

In [ ]:
for layer in new_model.layers:
    layer.trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [ ]:
new_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stopping = keras.callbacks.EarlyStopping(monitor='val_acc', patience=5, verbose=1, mode='auto')
callback_list = [early_stopping]

In [ ]:
new_model.fit(X_train,Y_train,epochs=10,batch_size=128,validation_data=(X_test,Y_test))

In [ ]:
BATCH_SIZE = 128
EPOCHS = 10

new_model.fit_generator(
    train_generator.flow(x_train, y_train, batch_size=BATCH_SIZE),
    steps_per_epoch=EPOCHS*len(x_train)/BATCH_SIZE,
    epochs=EPOCHS,
    verbose=1,
    shuffle=True,
    validation_data=value_generator.flow(x_test, y_test),
    validation_steps=EPOCHS*len(x_test)/BATCH_SIZE,
    callbacks=callback_list
)